In [1]:
import glob
import shutil
import os
import random


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# %matplotlib inline

In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome_train_test.zip', 'r')
zip_ref.extractall('/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome/')
zip_ref.close()

In [4]:
def folder_names(src_paths):
    src_paths = glob.glob(os.path.join(src_paths,'*'))
    l = []
    for path in src_paths:
        l.append(path.split("/")[-1])
    return l

In [5]:
def create_folder_at_destination(dest_path,folder_name_list):
    for folder_name in folder_name_list:
        os.mkdir(os.path.join(dest_path,folder_name))
    dest_folder_paths = glob.glob(os.path.join(dest_path,'*'))
    return dest_folder_paths


In [6]:
def train_test_images_genrator(src_path,dest_path,train_images,test_images,val_images):
    src_paths = os.path.join(src_path,"**")
    src_paths = glob.glob(src_paths)
    list_of_folders = ('train','val','test')

    dest_folder_paths = create_folder_at_destination(dest_path,list_of_folders)

    for path in dest_folder_paths:
        inside_train_folders = create_folder_at_destination(path,folder_names(src_path))

    final_folders_test = glob.glob(os.path.join(dest_folder_paths[0],'*'))
    final_folders_train =  glob.glob(os.path.join(dest_folder_paths[1],'*'))
    final_folders_val =  glob.glob(os.path.join(dest_folder_paths[2],'*'))


    folder_int = 0

    for src_dir in src_paths:
        jpg_images = glob.glob(os.path.join(src_dir,'*.jpg'))
        for i in random.sample(jpg_images, test_images):
            shutil.move(i, final_folders_test[folder_int])

        jpg_images = glob.glob(os.path.join(src_dir,'*.jpg'))
        for i in random.sample(jpg_images, train_images):
            shutil.move(i, final_folders_train[folder_int])

        jpg_images = glob.glob(os.path.join(src_dir,'*.jpg'))
        for i in random.sample(jpg_images, val_images):
            shutil.move(i, final_folders_val[folder_int])

        folder_int += 1

    print("Train and Test Folder created at : " + dest_path)


In [7]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome/rome_train_test/train'
valid_path = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome/rome_train_test/val'
test_path = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome/rome_train_test/test'

In [8]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=folder_names(train_path), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=folder_names(valid_path), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=folder_names(test_path), batch_size=10, shuffle=False)

Found 175 images belonging to 5 classes.
Found 35 images belonging to 5 classes.
Found 41 images belonging to 5 classes.


In [9]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [10]:
# VGG 16 architecture 

model = Sequential()


model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [11]:
model.add(Flatten())

model.add(Dense(units= 4096 , activation = "relu"))
model.add(Dense(units= 4096 , activation = "relu"))
model.add(Dense(units= 5 , activation = "softmax"))

In [12]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
# model.compile(optimizer=Adam(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

In [14]:
epochs=20
history = model.fit(train_batches,batch_size=10,validation_data=valid_batches,epochs=epochs)
# history = model.fit(x=train_batches,
#     steps_per_epoch=len(train_batches),
#     validation_data=valid_batches,
#     validation_steps=len(valid_batches),
#     epochs=20
# )

Epoch 1/20
18/18 [==============================] - 18s 328ms/step - loss: 2.0842 - accuracy: 0.1943 - val_loss: 1.5986 - val_accuracy: 0.2000
Epoch 2/20
18/18 [==============================] - 3s 191ms/step - loss: 1.5678 - accuracy: 0.2571 - val_loss: 1.4201 - val_accuracy: 0.4857
Epoch 3/20
18/18 [==============================] - 3s 191ms/step - loss: 1.5000 - accuracy: 0.4800 - val_loss: 1.4525 - val_accuracy: 0.3429
Epoch 4/20
18/18 [==============================] - 3s 192ms/step - loss: 1.2749 - accuracy: 0.5029 - val_loss: 0.9982 - val_accuracy: 0.6000
Epoch 5/20
18/18 [==============================] - 3s 192ms/step - loss: 1.0260 - accuracy: 0.6114 - val_loss: 0.9606 - val_accuracy: 0.6286
Epoch 6/20
18/18 [==============================] - 3s 192ms/step - loss: 0.9136 - accuracy: 0.6514 - val_loss: 1.1050 - val_accuracy: 0.5143
Epoch 7/20
18/18 [==============================] - 3s 193ms/step - loss: 1.0753 - accuracy: 0.5943 - val_loss: 1.0874 - val_accuracy: 0.5714
Epoch

In [15]:
print("predict on test data")
result_predict = model.predict(test_batches, batch_size=20)
print("test loss, test acc:", result_predict)

predict on test data
test loss, test acc: [[5.35139680e-01 6.73945341e-03 1.92203775e-01 1.20998817e-02
  2.53817230e-01]
 [9.97351527e-01 1.44609320e-03 6.19833532e-04 1.13695125e-04
  4.68829443e-04]
 [4.52394307e-01 1.88031478e-03 5.18183768e-01 3.61724687e-03
  2.39243358e-02]
 [9.59445894e-01 1.53622928e-03 3.62481736e-02 2.28599692e-03
  4.83658339e-04]
 [2.48405546e-01 9.66998559e-05 6.98420033e-02 1.21387164e-03
  6.80441856e-01]
 [9.91869211e-01 1.25146925e-03 8.44939670e-04 1.07086881e-03
  4.96351440e-03]
 [9.91370082e-01 5.37867332e-03 9.68303357e-05 2.36818427e-03
  7.86280492e-04]
 [8.36771905e-01 3.64481821e-03 1.96070615e-02 1.34999584e-02
  1.26476258e-01]
 [6.87378645e-02 5.04318625e-03 7.89594591e-01 1.35671079e-01
  9.53217444e-04]
 [7.29182223e-03 9.92041528e-01 1.21505727e-04 5.40088280e-04
  5.02623425e-06]
 [8.89128912e-03 9.90920305e-01 3.13801820e-06 1.84347969e-04
  9.00234397e-07]
 [6.94163144e-03 9.93040800e-01 1.38211863e-05 3.60886725e-06
  1.26773131e-07

In [16]:
print("Evaluate on test data")
result_evaluate = model.evaluate(test_batches, batch_size=20)
print("test loss, test acc:", result_evaluate)

Evaluate on test data
5/5 [==============================] - 0s 64ms/step - loss: 1.1425 - accuracy: 0.7317
test loss, test acc: [1.1425015926361084, 0.7317073345184326]


------

VGG 16 WITH AUGMENTED IMAGES

In [17]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome_aug.zip', 'r')
zip_ref.extractall('/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome/')
zip_ref.close()

KeyboardInterrupt: ignored

In [18]:
train_path_aug= '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome/rome_aug/train_aug'
valid_path_aug = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome/rome_aug/val_aug'
test_path_aug = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/rome/rome_aug/test_aug'

In [19]:
train_batches_aug = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path_aug, target_size=(224,224), classes=folder_names(train_path_aug), batch_size=10)
valid_batches_aug = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path_aug, target_size=(224,224), classes=folder_names(valid_path_aug), batch_size=10)
test_batches_aug = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path_aug, target_size=(224,224), classes=folder_names(test_path_aug), batch_size=10, shuffle=False)

Found 1750 images belonging to 5 classes.
Found 350 images belonging to 5 classes.
Found 410 images belonging to 5 classes.


In [20]:
model = Sequential()


model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [21]:
model.add(Flatten())

model.add(Dense(units= 4096 , activation = "relu"))
model.add(Dense(units= 4096 , activation = "relu"))
model.add(Dense(units= 5 , activation = "softmax"))

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                  

In [28]:
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
# model.compile(optimizer=Adam(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
epochs=20
history = model.fit(train_batches_aug,batch_size=200,validation_data=valid_batches_aug,epochs=epochs)

Epoch 1/20
  2/175 [..............................] - ETA: 23:06 - loss: 2.3260 - accuracy: 0.1500

KeyboardInterrupt: ignored

In [ ]:
print("Evaluate on test data")
result_evaluate_aug = model.evaluate(test_batches_aug, batch_size=20)
print("test loss, test acc:", result_evaluate_aug)

In [30]:
def plot_confusion_matrix(y_true, result_predict, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):

    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    #Compute confusion matrix
    cm = confusion_matrix(y_true, result_predict)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#print(cm)

    fig, ax = plt.subplots(figsize=(7,7))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')


    #Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

np.set_printoptions(precision=2)

In [31]:
confusion_mtx = confusion_matrix(np.array(y_test), np.argmax(result_predict, axis = -1))

NameError: ignored

In [32]:
class_names=['Cloudy', 'Foggy', 'Rainy', 'Snowy', 'Sunny']

In [33]:
plot_confusion_matrix(np.array(y_test), np.argmax(result_predict, axis = -1), classes = class_names, title='Confusion matrix, without normalization')

NameError: ignored

----------

VGG 16 ON NORMAL IMAGES WITH DROPOUT

In [34]:

model = Sequential()


model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(units= 4096 , activation = "relu"))
model.add(Dense(units= 4096 , activation = "relu"))
model.add(Dense(units= 5 , activation = "softmax"))


In [35]:
model.compile(optimizer=Adam(learning_rate=0.00001), 
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [36]:
epochs=50
history = model.fit(train_batches,batch_size=10,validation_data=valid_batches,epochs=epochs)

Epoch 1/50
18/18 [==============================] - 5s 219ms/step - loss: 1.6704 - accuracy: 0.2343 - val_loss: 1.5524 - val_accuracy: 0.3143
Epoch 2/50
18/18 [==============================] - 4s 212ms/step - loss: 1.5684 - accuracy: 0.2857 - val_loss: 1.5497 - val_accuracy: 0.2286
Epoch 3/50
15/18 [========================>.....] - ETA: 0s - loss: 1.5001 - accuracy: 0.4207

KeyboardInterrupt: ignored

In [ ]:
print("Evaluate on test data")
result_evaluate = model.evaluate(test_batches, batch_size=20)
print("test loss, test acc:", result_evaluate)

Evaluate on test data
5/5 [==============================] - 1s 267ms/step - loss: 0.9081 - accuracy: 0.7561
test loss, test acc: [0.9081196784973145, 0.7560975551605225]


--------------

VGG 16 Using dropout layer


In [37]:
model = Sequential()


model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.3))


model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(units= 4096 , activation = "relu"))
model.add(Dense(units= 4096 , activation = "relu"))
model.add(Dense(units= 5 , activation = "softmax"))

In [38]:
model.compile(optimizer=Adam(learning_rate=0.00001), 
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [39]:
epochs=50
history = model.fit(train_batches_aug,batch_size=10,validation_data=valid_batches_aug,epochs=epochs)

Epoch 1/50
 66/175 [==========>...................] - ETA: 10:15 - loss: 1.7116 - accuracy: 0.2167

UnknownError: ignored